<a href="https://colab.research.google.com/github/mfernl/03MIAR_Algoritmos-de-optimizacion-2026/blob/main/Trabajo_practico_Tribunales_TFM_mfernl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Marco Fernández Llamas  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---/tree/master/TrabajoPractico<br>
Google Colab: https://colab.research.google.com/drive/10vcf6mztWS27BvXNkXRYOVHwaeoioQCq?usp=sharing<br>
Problema:
Configuración de Tribunales

Descripción del problema:
Se precisa configurar tribunales de evaluación para un grupo de 15 alumnos que desean presentar su Trabajo Fin de Máster (TFM).
Cada tribunal está compuesto por tres profesores, cada uno desempeñando uno de los siguientes roles: Presidente, Secretario o Vocal.
Los profesores han indicado su disponibilidad horaria para participar en los tribunales de 15h a 21h durante la semana del 15 al 19 de abril.

Hay 15 alumnos, por lo que se deben configurar 15 tribunales buscando la configuración más equilibrada posible. Además ningún profesor puede ser convocado a dos tribunales al mismo tiempo ni a un tribunal al que no tenga disponibilidad.






                                        

#Implementación de la solución

In [126]:
import io
import pandas as pd
import numpy as np

disponibilidad = pd.read_csv(io.StringIO('''
0,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0
1,1,1,1,0,0,0,0,1,1,1,1,0,0,1,0,0,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1
0,0,1,1,0,1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,0,1,0,1,1,0,1,0,1
1,0,1,0,1,1,0,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,0,1,1,0,1,1,1,1,1,1,1,1,0
1,1,0,1,0,1,1,1,1,1,0,1,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,0
1,1,1,1,1,0,0,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,0,0,1,1,1,1,1,0,1,0,1
0,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,1,1,0,0,1,1,0,0,0,0,1,1,1,1,1,1,1,0,1
1,1,1,1,1,0,0,1,1,0,1,1,1,0,1,1,1,0,0,1,1,0,1,1,1,1,1,1,0,1,1,1,0,1,0
1,0,1,1,0,1,0,0,1,1,1,1,1,1,1,1,0,0,0,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1
1,1,0,1,1,0,1,1,0,0,0,0,0,1,1,1,0,0,1,1,1,1,0,0,1,1,1,1,1,1,1,0,0,0,1
'''), names=['15-15','15-16','15-17','15-18','15-19','15-20','15-21','16-15','16-16','16-17','16-18','16-19','16-20','16-21','17-15','17-16','17-17','17-18','17-19','17-20','17-21','18-15','18-16','18-17','18-18','18-19','18-20','18-21','19-15','19-16','19-17','19-18','19-19','19-20','19-21',], )
disponibilidad.index = ['RRD','QYV','LHL','HLC','MSB','PMQ','QWF','EBB','IOE','IOA']

roles = pd.read_csv(io.StringIO('''
Profesor,ROL
RRD,"P,S,V"
QYV,"P,S,V"
LHL,"P,V"
HLC,"S,V"
MSB,"P,S,V"
PMQ,"P,S,V"
QWF,"S,V"
EBB,"S,V"
IOE,"P,S,V"
IOA,"P,S,V"
'''),header = 0)

sol = pd.DataFrame(data = np.zeros((15,4)),columns = ['Fecha-Hora', 'Presidente', 'Secretario', 'Vocal'], dtype='string')

In [127]:
display(disponibilidad)
display(roles)
display(sol)

,15-15,15-16,15-17,15-18,15-19,15-20,15-21,16-15,16-16,16-17,...,18-19,18-20,18-21,19-15,19-16,19-17,19-18,19-19,19-20,19-21
RRD,0,1,1,1,0,1,1,1,0,1,...,1,1,1,1,1,1,1,1,0,0
QYV,1,1,1,1,0,0,0,0,1,1,...,1,1,1,1,1,1,1,1,1,1
LHL,0,0,1,1,0,1,1,1,1,1,...,1,0,1,0,1,1,0,1,0,1
HLC,1,0,1,0,1,1,0,1,0,0,...,0,1,1,1,1,1,1,1,1,0
MSB,1,1,0,1,0,1,1,1,1,1,...,0,1,1,1,0,1,1,1,1,0
PMQ,1,1,1,1,1,0,0,1,1,1,...,0,1,1,1,1,1,0,1,0,1
QWF,0,1,1,1,1,1,1,1,1,0,...,0,1,1,1,1,1,1,1,0,1
EBB,1,1,1,1,1,0,0,1,1,0,...,1,1,1,0,1,1,1,0,1,0
IOE,1,0,1,1,0,1,0,0,1,1,...,1,1,0,1,0,1,1,1,1,1
IOA,1,1,0,1,1,0,1,1,0,0,...,1,1,1,1,1,1,0,0,0,1


,Profesor,ROL
0,RRD,"P,S,V"
1,QYV,"P,S,V"
2,LHL,"P,V"
3,HLC,"S,V"
4,MSB,"P,S,V"
5,PMQ,"P,S,V"
6,QWF,"S,V"
7,EBB,"S,V"
8,IOE,"P,S,V"
9,IOA,"P,S,V"


,Fecha-Hora,Presidente,Secretario,Vocal
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0


In [132]:
def búsqueda_profundidad(disp, roles, sol):
    sol_copy = sol.copy()
    profesores = disp.index
    horas_disp = disp.columns
    trib = 0

    for i in range(len(horas_disp)):
        current_time_slot = horas_disp[i]
        for j in range(len(profesores)):
          current_profesor = profesores[j]
          if(trib == 15):
            print("Tribunales completos")
            return sol_copy
          else:
            if disp.iloc[j,i] == 1 and sol_copy.loc[trib,'Presidente'] == '0.0': # Si el profesor está disponible en la franja, y no hay presidente:

              professor_roles_str = roles.loc[roles['Profesor'] == current_profesor]['ROL'].iloc[0]

              if 'P' in professor_roles_str: #Si puede ser presidente
                print(f'SOy presi: {current_profesor} en {current_time_slot}')
                sol_copy.loc[trib, 'Presidente'] = current_profesor #Se asigna como presidente

            elif disp.iloc[j,i] == 1 and sol_copy.loc[trib,'Secretario'] == '0.0': # Si el profesor está disponible en la franja, y no hay secretario:

              professor_roles_str = roles.loc[roles['Profesor'] == current_profesor]['ROL'].iloc[0]

              if 'S' in professor_roles_str: #Si puede ser secretario
                print(f'SOy secretario: {current_profesor} en {current_time_slot}')
                sol_copy.loc[trib, 'Secretario'] = current_profesor #Se asigna como secretario

            elif disp.iloc[j,i] == 1 and sol_copy.loc[trib,'Vocal'] == '0.0': # Si el profesor está disponible en la franja, y no hay vocal:

              professor_roles_str = roles.loc[roles['Profesor'] == current_profesor]['ROL'].iloc[0]

              if 'V' in professor_roles_str: #Si puede ser vocal
                print(f'SOy vocal: {current_profesor} en {current_time_slot}')
                sol_copy.loc[trib, 'Vocal'] = current_profesor #Se asigna como vocal

              if sol_copy.loc[trib,'Presidente'] != '0.0' and sol_copy.loc[trib,'Secretario'] != '0.0' and sol_copy.loc[trib,'Vocal'] != '0.0': #tribunal completo (pasar a función)
                sol_copy.loc[trib, 'Fecha-Hora'] = current_time_slot
                trib += 1
                print("Tribunal completo: " + current_time_slot)
                break
        else:
          continue


In [133]:
s = búsqueda_profundidad(disponibilidad,roles,sol)
display(s)

SOy presi: QYV en 15-15
SOy secretario: HLC en 15-15
SOy vocal: MSB en 15-15
Tribunal completo: 15-15
SOy presi: RRD en 15-16
SOy secretario: QYV en 15-16
SOy vocal: MSB en 15-16
Tribunal completo: 15-16
SOy presi: RRD en 15-17
SOy secretario: QYV en 15-17
SOy vocal: LHL en 15-17
Tribunal completo: 15-17
SOy presi: RRD en 15-18
SOy secretario: QYV en 15-18
SOy vocal: LHL en 15-18
Tribunal completo: 15-18
SOy presi: PMQ en 15-19
SOy secretario: QWF en 15-19
SOy vocal: EBB en 15-19
Tribunal completo: 15-19
SOy presi: RRD en 15-20
SOy secretario: HLC en 15-20
SOy vocal: MSB en 15-20
Tribunal completo: 15-20
SOy presi: RRD en 15-21
SOy secretario: MSB en 15-21
SOy vocal: QWF en 15-21
Tribunal completo: 15-21
SOy presi: RRD en 16-15
SOy secretario: HLC en 16-15
SOy vocal: MSB en 16-15
Tribunal completo: 16-15
SOy presi: QYV en 16-16
SOy secretario: MSB en 16-16
SOy vocal: PMQ en 16-16
Tribunal completo: 16-16
SOy presi: RRD en 16-17
SOy secretario: QYV en 16-17
SOy vocal: LHL en 16-17
Tribu

,Fecha-Hora,Presidente,Secretario,Vocal
0,15-15,QYV,HLC,MSB
1,15-16,RRD,QYV,MSB
2,15-17,RRD,QYV,LHL
3,15-18,RRD,QYV,LHL
4,15-19,PMQ,QWF,EBB
5,15-20,RRD,HLC,MSB
6,15-21,RRD,MSB,QWF
7,16-15,RRD,HLC,MSB
8,16-16,QYV,MSB,PMQ
9,16-17,RRD,QYV,LHL


#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

In [ ]:
#Respuesta
'''

'''


#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

In [ ]:
#Respuesta


#Diseño
- ¿Que técnica utilizo? ¿Por qué?

In [ ]:
#Respuesta Algoritmo VORAZ